In [1]:
import glob
import datetime
import pandas as pd

In [2]:
momentumfiles=glob.glob('onlymomentum_data_*_szokereso_result.csv')

In [3]:
dates=\
[datetime.datetime(int(each[18:22]), int(each[23:25]), int(each[26:28]))
 for each in momentumfiles]

In [4]:
def day(date):
    return date.weekday()
def week(date):
    return date.isocalendar()[1]

In [5]:
df = pd.DataFrame(zip(momentumfiles,dates),columns=['momentumfiles','dates'])

In [6]:
df['dayint'] =   df['dates'].apply(day )
df['isoweeknum']=df['dates'].apply(week)

In [7]:
df.sort_values(by='isoweeknum',inplace=True)

In [8]:
dfsByWeek = {'week'+str(isoweeknum):\
                         df[df['isoweeknum']==isoweeknum].reset_index(drop=True)
                         for isoweeknum in list(set(df['isoweeknum']))}

In [9]:
dfsByWeekUnified ={week:pd.concat([pd.read_csv(file)
                        for file in dfsByWeek[week]['momentumfiles']])
                        for week in dfsByWeek.keys()}

In [10]:
weeklyFreqDf = {week:pd.concat([dfsByWeekUnified[week][col]
                 for col in dfsByWeekUnified[week].columns if 'momentum' in col]
               ).value_counts(
               ).rename_axis('unique_entries'
               ).reset_index(name='counts')
                 for week in dfsByWeekUnified.keys()}

In [11]:
dfsByDay = {str(date)[:10]:df[df['dates']==date] for date in list(set(df['dates']))}

In [12]:
dfsByDayUnified ={day:pd.concat([pd.read_csv(file)
                     for file in dfsByDay[day]['momentumfiles']])
                     for day in dfsByDay.keys()}

In [13]:
dailyFreqDf = {day:pd.concat([dfsByDayUnified[day][col]
                 for col in dfsByDayUnified[day].columns if 'momentum' in col]
               ).value_counts(
               ).rename_axis('unique_entries'
               ).reset_index(name='counts')
                 for day in dfsByDayUnified.keys()}

In [14]:
for eachDay in dailyFreqDf.keys():
    dailyFreqDf[eachDay].to_csv(
        '/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/dailyfreqlist/'+
        'freqlist_'+eachDay+'.csv')

In [15]:
for eachWeek in weeklyFreqDf.keys():
    weeklyFreqDf[eachWeek].to_csv(
        '/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/weeklyfreqlist/'+
        'freqlist_'+eachWeek+'.csv')